In [2]:
import pandas as pd 
import numpy as np

In [3]:
data = pd.read_csv('hadoop.csv')

In [4]:
data = data.drop(['idx','access_date','access_time','access_page','access_referer','item_name'],axis=1)

In [5]:
unique_ip = data[data['item_type']=='cart']['access_ip'].unique()

## Make Dict_ip insert key is ip item is values

In [6]:
dict_ip = dict.fromkeys(unique_ip)   #dict key 생성

for i in dict_ip.keys():
    dict_ip[i] = []

#-----------------------------------------------#

for i in range(len(data)):          #dict value 삽입
    if (data['access_ip'].iloc[i] in dict_ip.keys()):
        key = data['access_ip'].iloc[i]
        dict_ip[key].append(data['item_id'].iloc[i])

In [7]:
for i,j in dict_ip.items():
    dict_ip[i] = list(map(str,j))        #dict value를 string으로 매핑
    
#------------------------------------------------#
#dict의 벨류가 4 미만이면 삭제
dict_ip = {key:value for key, value in dict_ip.items() if len(value) >4} 

In [8]:
cart_item = []       #dict의 마지막 아이템을 카트 아이템배열에 넣음
for i,j in dict_ip.items():
    cart_item.append(j[-1])

In [9]:
columns_name = []      #column이름을 string으로 변환
for i in range(14):
    columns_name.append(str(i))

## Make DataFrame New

In [10]:
df = pd.DataFrame.from_dict(dict_ip,orient='index',columns=columns_name)
df['13'] = cart_item

In [11]:
mask = df.applymap(lambda x: x is None)         #None값들 삭제
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''

In [12]:
item_list = []                 #item들을 뽑아서 한 row씩 한 리스트로 묶어 배열 생성
for i in range(len(df)):
    item_list.append(df.iloc[i,:].tolist())

# Word2Vec

In [13]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [14]:
path = get_tmpfile('word2vec.model')

In [15]:
model = Word2Vec(item_list, size=2, window=5, min_count=1,workers=4)
model.save('word2vec.model')

In [16]:
model = Word2Vec.load('word2vec.model')

In [17]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 

In [18]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [19]:
# def plot_2d_graph(vocabs, xs, ys):
#     plt.figure(figsize=(12,6))
#     plt.scatter(xs,ys,marker='o')
#     for i, v in enumerate(vocabs):
#         plt.annotate(v,xy=(xs[i],ys[i]))

In [20]:
#pca = PCA(n_components=2)

In [21]:
# xys = pca.fit_transform(word_vectors_list)
# xs = xys[0]
# ys = xys[1]

In [22]:
# plot_2d_graph(vocabs,xs,ys)

In [23]:
for i in range(len(df)):            #word2vec value를 df로 넣음
    for j in range(len(df.iloc[i])):
        key=str(df.iloc[i,j])
        df.iloc[i,j] = word_vectors[key].tolist()

In [24]:
df.index = range(len(df))

# Data split && Learning

In [86]:
from sklearn import preprocessing
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [87]:
X_data = df.drop('13',axis=1)
y_data = df['13']

In [124]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,"[-0.1640259027481079, 4.508819103240967]","[-0.1588466614484787, 6.878407955169678]","[-0.16196171939373016, 7.150777816772461]","[-0.1588466614484787, 6.878407955169678]","[-0.22434669733047485, 7.150781631469727]","[-0.1588466614484787, 6.878407955169678]","[-0.2067328542470932, 7.016667366027832]","[-0.16196171939373016, 7.150777816772461]","[-0.22434669733047485, 7.150781631469727]","[-0.22434669733047485, 7.150781631469727]","[-0.1588466614484787, 6.878407955169678]","[-0.2067328542470932, 7.016667366027832]","[-0.22434669733047485, 7.150781631469727]","[-0.22434669733047485, 7.150781631469727]"
1,"[5.086978912353516, 0.01426604948937893]","[8.350432395935059, 0.07338181883096695]","[8.350432395935059, 0.07338181883096695]","[8.530829429626465, 0.0792444720864296]","[8.350432395935059, 0.07338181883096695]","[8.467345237731934, 0.06705528497695923]","[8.530829429626465, 0.0792444720864296]","[8.530829429626465, 0.0792444720864296]","[8.467345237731934, 0.06705528497695923]","[8.467345237731934, 0.06705528497695923]","[8.467345237731934, 0.06705528497695923]","[8.467345237731934, 0.06705528497695923]","[8.350432395935059, 0.07338181883096695]","[8.350432395935059, 0.07338181883096695]"
2,"[1.7177468538284302, 1.3141669034957886]","[1.1859112977981567, 1.077933430671692]","[0.890296459197998, 0.8472775220870972]","[0.9072114825248718, 1.0453906059265137]","[1.4925833940505981, 1.4037553071975708]","[1.11690092086792, 1.069031834602356]","[1.3312113285064697, 1.0163538455963135]","[1.153890609741211, 0.9664674401283264]","[1.4144071340560913, 1.145633339881897]","[0.896998405456543, 1.1110385656356812]","[0.9875347018241882, 1.065385341644287]","[0.9954778552055359, 0.9734124541282654]","[1.1252695322036743, 1.129090666770935]","[1.1252695322036743, 1.129090666770935]"
3,"[1.7177468538284302, 1.3141669034957886]","[0.9864791035652161, 1.0006539821624756]","[1.0449612140655518, 0.8524702191352844]","[0.8948123455047607, 0.8786210417747498]","[0.8558868169784546, 0.6732153296470642]","[0.6789243221282959, 0.7632874250411987]","[1.3536882400512695, 1.5095099210739136]","[1.3536882400512695, 1.5095099210739136]","[0.8135942816734314, 0.8362913727760315]","[0.984010636806488, 0.8045843839645386]","[1.412858009338379, 1.3378605842590332]","[1.088322639465332, 1.006712555885315]","[0.8800769448280334, 0.8567090034484863]","[0.8800769448280334, 0.8567090034484863]"
4,"[0.04985416308045387, 4.521084308624268]","[0.03289095684885979, 6.5604777336120605]","[0.03289095684885979, 6.5604777336120605]","[0.008788080886006355, 6.2681121826171875]","[0.053544338792562485, 6.506516933441162]","[0.05795108154416084, 6.412958145141602]","[0.05795108154416084, 6.412958145141602]","[0.00477069616317749, 6.455761909484863]","[0.04595549404621124, 6.339162349700928]","[0.007962689734995365, 6.611814975738525]","[0.053544338792562485, 6.506516933441162]","[0.00477069616317749, 6.455761909484863]","[0.03289095684885979, 6.5604777336120605]","[0.03289095684885979, 6.5604777336120605]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,"[1.3536882400512695, 1.5095099210739136]","[1.3536882400512695, 1.5095099210739136]","[1.4886810779571533, 1.3893399238586426]","[1.4886810779571533, 1.3893399238586426]","[1.446107029914856, 1.5721988677978516]","[1.3874143362045288, 1.4449114799499512]","[-0.22434669733047485, 7.150781631469727]","[-0.22434669733047485, 7.150781631469727]","[0.4945278465747833, 0.7728760242462158]","[0.4945278465747833, 0.7728760242462158]","[0.4945278465747833, 0.7728760242462158]","[0.4945278465747833, 0.7728760242462158]","[0.4945278465747833, 0.7728760242462158]","[-0.22434669733047485, 7.150781631469727]"
6964,"[1.0347899198532104, 1.1084198951721191]","[1.1006349325180054, 1.152884840965271]","[1.0682339668273926, 1.1513891220092773]","[1.1917595863342285, 1.2086092233657837]","[1.1050615310668945, 1.0681443214416504]","[1.1050615310668945, 1.0681443214416504]","[0.49452784657478

In [126]:
x_vector = []

for i in range(len(df)):
    x_vector.append(df.iloc[i])

In [127]:
x_vector

[0      [-0.1640259027481079, 4.508819103240967]
 1      [-0.1588466614484787, 6.878407955169678]
 2     [-0.16196171939373016, 7.150777816772461]
 3      [-0.1588466614484787, 6.878407955169678]
 4     [-0.22434669733047485, 7.150781631469727]
 5      [-0.1588466614484787, 6.878407955169678]
 6      [-0.2067328542470932, 7.016667366027832]
 7     [-0.16196171939373016, 7.150777816772461]
 8     [-0.22434669733047485, 7.150781631469727]
 9     [-0.22434669733047485, 7.150781631469727]
 10     [-0.1588466614484787, 6.878407955169678]
 11     [-0.2067328542470932, 7.016667366027832]
 12    [-0.22434669733047485, 7.150781631469727]
 13    [-0.22434669733047485, 7.150781631469727]
 Name: 0, dtype: object,
 0     [5.086978912353516, 0.01426604948937893]
 1     [8.350432395935059, 0.07338181883096695]
 2     [8.350432395935059, 0.07338181883096695]
 3      [8.530829429626465, 0.0792444720864296]
 4     [8.350432395935059, 0.07338181883096695]
 5     [8.467345237731934, 0.06705528497695923]
 

In [120]:
y_vector = []
for i in range(len(word_vectors_list)):
    y_vector.append(word_vectors_list[i][1])

In [121]:
y_vector

[-4.082396,
 -6.4118123,
 -6.5322304,
 -6.513954,
 -6.3384457,
 -1.730732,
 -2.8330789,
 -2.798418,
 -2.7716885,
 -2.6972704,
 -1.5375861,
 -1.0396564,
 -1.4738913,
 -2.0365837,
 -1.4030126,
 -1.6855,
 -1.4727561,
 -1.806158,
 -1.2606555,
 -1.323808,
 -1.4229417,
 -1.5502635,
 -1.5532119,
 -1.1838574,
 -1.2544943,
 -1.1652646,
 -1.0232673,
 -2.5823188,
 -1.0960538,
 -1.2709951,
 -2.4846616,
 -1.5326397,
 -1.3361838,
 -1.8121982,
 -2.5413444,
 -2.467466,
 -2.4428906,
 -2.5152187,
 -2.4444132,
 -2.468663,
 -2.5255313,
 -1.2256812,
 -2.1030679,
 -2.1023104,
 -1.8124126,
 -2.4974098,
 -2.565759,
 -2.5044549,
 -2.4178338,
 -2.5544386,
 -2.4988987,
 -1.4932418,
 -1.0901642,
 -1.6857755,
 -1.6178224,
 -1.5093212,
 -1.4706203,
 -1.7598898,
 -1.3302324,
 -1.8726009,
 -1.5573112,
 -1.3138845,
 -1.2851119,
 -3.0118022,
 -1.2270805,
 -1.2520802,
 -1.4219141,
 -6.633173,
 -0.6899991,
 -1.3552929,
 -1.8079079,
 -1.0768012,
 -1.181059,
 -1.5508366,
 -1.8115441,
 -1.5137355,
 -1.0666529,
 -1.1447966,


In [29]:
seed = 7
test_size = 0.3
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size,  random_state=seed)

In [60]:
xgb.XGBClassifier.fit(X_data,y_data)

TypeError: fit() missing 1 required positional argument: 'y'

In [ ]:
# model = xgb.XGBClassifier(n_estimators=100,max_deplearning_rate=0.1,subsample=0.5,eval_metric=eval_metric,verbose=True)

In [ ]:
# model.fit(X_train,y_train,eval_set=eval_set, eval_metric=eval_metric,verbose=True)

In [ ]:
model = XGBClassifier()
eval_set=[(X_train,y_train),(X_test,y_test)]

In [ ]:
model.fit(X_train,y_train,eval_metric=['error','logloss'],eval_set=eval_set)

In [ ]:
a = model.evals_result()

In [ ]:
result = pd.DataFrame()

In [ ]:
prediction = model.predict(X_test)

In [ ]:
result['X_test'] = prediction
result['Y_test'] = y_test.tolist()

In [ ]:
result

In [ ]:
X_train.index = range(len(X_train))

In [ ]:
y_train.index = range(len(y_train))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# confusion_matrix(prediction,y_test)

In [ ]:
X_train

In [ ]:
result.head(30)

# Test